In [4]:

!pip install xgboost scikit-learn pandas pyarrow joblib

   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
    --------------------------------------- 1.8/101.7 MB 11.2 MB/s eta 0:00:09
   - -------------------------------------- 4.2/101.7 MB 11.4 MB/s eta 0:00:09
   -- ------------------------------------- 6.3/101.7 MB 11.0 MB/s eta 0:00:09
   --- ------------------------------------ 8.4/101.7 MB 10.8 MB/s eta 0:00:09
   ---- ----------------------------------- 10.7/101.7 MB 11.0 MB/s eta 0:00:09
   ----- ---------------------------------- 13.4/101.7 MB 11.0 MB/s eta 0:00:09
   ----- ---------------------------------- 14.9/101.7 MB 10.6 MB/s eta 0:00:09
   ------ --------------------------------- 16.3/101.7 MB 10.3 MB/s eta 0:00:09
   ------- -------------------------------- 17.8/101.7 MB 9.9 MB/s eta 0:00:09
   ------- -------------------------------- 19.1/101.7 MB 9.5 MB/s eta 0:00:09
   -------- ------------------------------- 20.7/101.7 MB 9.3 MB/s eta 0:00:09
   -------- ------------------------------- 22.3/101.7 M

In [5]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import xgboost as xgb

print("✅ Imports successful")

✅ Imports successful


In [14]:
PROJECT_ROOT = r"C:\Users\Lavanya\Desktop\AI-Phishing-"
os.chdir(PROJECT_ROOT)
print(f"Working directory: {os.getcwd()}")

Working directory: C:\Users\Lavanya\Desktop\AI-Phishing-


In [8]:
!pip install fastparquet

   ---------------------------------------- 0.0/669.7 kB ? eta -:--:--
   ---------------------------------------- 669.7/669.7 kB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 10.3 MB/s eta 0:00:00

   -------------------- ------------------- 1/2 [fastparquet]
   ---------------------------------------- 2/2 [fastparquet]



In [9]:
df = pd.read_parquet(
    r"C:\Users\Lavanya\Desktop\AI-Phishing-\datasets\processed\df_clean.parquet.gzip",
    engine='fastparquet'
)

In [15]:
train_idx = joblib.load(r"C:\Users\Lavanya\Desktop\AI-Phishing-\datasets\processed\train_idx.pkl")
val_idx   = joblib.load(r"C:\Users\Lavanya\Desktop\AI-Phishing-\datasets\processed\val_idx.pkl")
test_idx  = joblib.load(r"C:\Users\Lavanya\Desktop\AI-Phishing-\datasets\processed\test_idx.pkl")

feature_cols = ['url_count', 'url_length_max', 'url_subdom_max', 'attachment_count']

X_train = df.loc[train_idx, feature_cols].fillna(0)
X_val   = df.loc[val_idx,   feature_cols].fillna(0)
X_test  = df.loc[test_idx,  feature_cols].fillna(0)

y_train = df.loc[train_idx, 'label']
y_val   = df.loc[val_idx,   'label']
y_test  = df.loc[test_idx,  'label']

texts_train = df.loc[train_idx, 'text']
texts_val   = df.loc[val_idx,   'text']
texts_test  = df.loc[test_idx,  'text']

print(f"✅ Loaded: Train={len(X_train)}, Val={len(X_val)}, Test={len(X_test)}")

✅ Loaded: Train=86947, Val=10868, Test=10869


In [11]:
model_url_path = r"models\xgboost_model.json"
import os
os.makedirs("models", exist_ok=True)

if os.path.exists(model_url_path):
    model_url = xgb.XGBClassifier()
    model_url.load_model(model_url_path)
    print("✅ Loaded existing URL XGBoost model.")
else:
    neg, pos = np.bincount(y_train)
    scale = neg / pos
    model_url = xgb.XGBClassifier(
        n_estimators=200,
        max_depth=4,
        learning_rate=0.1,
        scale_pos_weight=scale,
        eval_metric='logloss',
        use_label_encoder=False,
        random_state=42
    )
    model_url.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    model_url.save_model(model_url_path)
    print("✅ Trained and saved new URL XGBoost model.")

y_pred_url = model_url.predict(X_test)
from sklearn.metrics import classification_report
print("\n🔹 URL XGBoost Performance:")
print(classification_report(y_test, y_pred_url))

✅ Loaded existing URL XGBoost model.

🔹 URL XGBoost Performance:
              precision    recall  f1-score   support

           0       0.83      0.83      0.83      6065
           1       0.78      0.78      0.78      4804

    accuracy                           0.81     10869
   macro avg       0.81      0.81      0.81     10869
weighted avg       0.81      0.81      0.81     10869



In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1,2), stop_words='english')
X_text_train = vectorizer.fit_transform(texts_train)
X_text_val   = vectorizer.transform(texts_val)
X_text_test  = vectorizer.transform(texts_test)

neg, pos = np.bincount(y_train)
scale = neg / pos

model_text = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    scale_pos_weight=scale,
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42,
    tree_method='hist',
    n_jobs=-1
)

model_text.fit(X_text_train, y_train,
               eval_set=[(X_text_val, y_val)],
               verbose=False)

y_pred_text = model_text.predict(X_text_test)
print("\n🔹 Text XGBoost (TF-IDF) Performance:")
print(classification_report(y_test, y_pred_text))

joblib.dump(vectorizer, r"models\tfidf_vectorizer.pkl")
model_text.save_model(r"models\xgboost_text_model.json")
print("✅ TF-IDF vectorizer and text XGBoost saved.")

C:\Users\Lavanya\anaconda3\lib\site-packages\xgboost\training.py:200: UserWarning: [19:53:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:782: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



🔹 Text XGBoost (TF-IDF) Performance:
              precision    recall  f1-score   support

           0       0.97      0.96      0.97      6065
           1       0.95      0.97      0.96      4804

    accuracy                           0.96     10869
   macro avg       0.96      0.96      0.96     10869
weighted avg       0.96      0.96      0.96     10869

✅ TF-IDF vectorizer and text XGBoost saved.


In [13]:
from sklearn.linear_model import LogisticRegression

p_url_val = model_url.predict_proba(X_val)[:, 1]
p_text_val = model_text.predict_proba(X_text_val)[:, 1]

meta_X = np.column_stack([p_url_val, p_text_val])
meta_y = y_val.values
meta_model = LogisticRegression()
meta_model.fit(meta_X, meta_y)

joblib.dump(meta_model, r"models\meta_learner.pkl")

p_url_test = model_url.predict_proba(X_test)[:, 1]
p_text_test = model_text.predict_proba(X_text_test)[:, 1]
meta_X_test = np.column_stack([p_url_test, p_text_test])
final_preds = meta_model.predict(meta_X_test)

print("\n🔹 ENSEMBLE (URL + Text) PERFORMANCE:")
print(classification_report(y_test, final_preds))


🔹 ENSEMBLE (URL + Text) PERFORMANCE:
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      6065
           1       0.97      0.96      0.96      4804

    accuracy                           0.97     10869
   macro avg       0.97      0.97      0.97     10869
weighted avg       0.97      0.97      0.97     10869

